In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#from sklearn.externals import joblib
#xgbc0 = joblib.load('./models/xgbc0.m')

In [ ]:
imp_feat = pd.read_csv('imp_features.csv')
lists = imp_feat['featimp']

In [ ]:
X_train = pd.read_csv('train_total1.csv', usecols = lists)
y = pd.read_csv('train_total1.csv', usecols = ['orderlabel'])
y_train = y['orderlabel']

In [ ]:
#data = pd.read_csv('train_total1.csv')

In [ ]:
from sklearn.cross_validation import cross_val_score

In [ ]:
from sklearn.cross_validation import cross_val_score
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.grid_search import GridSearchCV

In [ ]:
xgbc1 = XGBClassifier()

In [ ]:
print (cross_val_score(xgbc1, X_train, y_train, cv=5, scoring='roc_auc').mean())

In [ ]:
target = 'orderlabel'
predictors = [x for x in data.columns if x not in ['orderlabel']]
def modelfit(alg, dtrain, predictors, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds, metrics='auc',early_stopping_rounds=early_stopping_rounds) #, show_progress=False
        alg.set_params(n_estimators=cvresult.shape[0])
        
    alg.fit(dtrain[predictors], dtrain[target], eval_metric='auc')
    
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
    
    print ('\nModel Report')
    print (metrics.accuracy_score(dtrain[target].values, dtrain_predictions)) # Accuracy
    print (metrics.roc_auc_score(dtrain[target], dtrain_predprob)) #'AUC Score (Train): %f') 
    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    print (feat_imp) 
    #plt.ylabel('Feature Importance Score')

用分好的部分值作为训练集。

In [ ]:
data = pd.read_csv('./partsample/X_train_p2.csv')

In [ ]:
modelfit(xgbc1, data, predictors)

Model Report
0.954323824993
0.98297650585

In [ ]:
xgbc2=XGBClassifier(
learning_rate = 0.05,
n_estimators = 1500,
max_depth = 5,
min_child_weight = 3,
gamma = 0,
subsample = 0.8,
colsample_bytree = 0.8,
objective = 'binary:logistic',
nthread = -1,
scale_pos_weight = 1,
reg_alpha=0.005,
seed = 27)

0.92826

以上跑了20分钟，所以估计以下参数需要4个小时。
XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)


Model Report
0.974094125198
0.991553588279

2小时40分钟

In [ ]:
param_test1 = {
    'max_depth':list(range(3,10,2)),
    'min_child_weight':list(range(1,6,2))
}
gsearch1 = GridSearchCV(estimator = XGBClassifier (
learning_rate = 0.1,
n_estimators = 140,
max_depth = 5,
min_child_weight = 1,
gamma = 0,
subsample = 0.8,
colsample_bytree = 0.8,
objective = 'binary:logistic',
nthread = -1,
scale_pos_weight = 1,
seed = 27
), param_grid = param_test1, scoring='roc_auc', n_jobs=-1, iid=False, cv=5, verbose=1)

python3里面，range返回的是一个迭代值，用list（）转化后可以和2一样使用

In [ ]:
param_test1 = {
    'max_depth':list(range(3,10,2)),
    'min_child_weight':list(range(1,6,2))
}
xgbc1 = XGBClassifier (
learning_rate = 0.1,
n_estimators = 140,
max_depth = 5,
min_child_weight = 1,
gamma = 0,
subsample = 0.8,
colsample_bytree = 0.8,
objective = 'binary:logistic',
nthread = -1,
scale_pos_weight = 1,
seed = 27
)
gsearch1 = GridSearchCV(xgbc1, param_test1, scoring='roc_auc', n_jobs=-1, iid=False, cv=5, verbose=1)

In [ ]:
gsearch1.fit(Dtrain[predictors], Dtrain[target])
gsearch1.grid_scores_
gsearch1.best_params_
gsearch1.best_score_

In [ ]:
X_test = pd.read_csv('test_total.csv', usecols = predictors)
label = ['orderid','roomid']
X_sub = pd.read_csv('competition_test.txt', sep='\t', usecols = label)
y_pred = xgbc2.predict_proba(X_test)[:,1]

In [ ]:
X_sub['predict_roomid'] = X_sub['roomid']
X_sub['pred'] = y_pred
X_sub = X_sub.drop(['roomid'], axis=1)

In [ ]:
subsub = X_sub.sort_index(by=['orderid', 'pred'], ascending=False)
sub_xgbc_final = subsub.groupby(subsub['orderid']).head(1)
sub_xgbc_final = sub_xgbc_final.drop(['pred'],axis=1)
sub_xgbc_final.to_csv('./final_xgbc.csv', index=False)